In [1]:
from pathlib import Path

import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

In [4]:
cwd = Path.cwd()
tracks_dir = cwd / 'tracks'
len(list(tracks_dir.iterdir()))

293

In [6]:
connection_args = {
    'host': 'localhost',
    'dbname': 'names',
#     'port': 5432
}

connection = pg.connect(**connection_args)

In [7]:
cursor = connection.cursor()